### Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [2]:
df = pd.read_csv("C:\\Users\\AJIT\\Desktop\\santander-customer-transaction-prediction\\train.csv")
df_test = pd.read_csv("C:\\Users\\AJIT\\Desktop\\santander-customer-transaction-prediction\\test.csv")

In [3]:
df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
df.shape

(200000, 202)

In [3]:
df['Type']='Train'
df_test['Type']='Test'
data = pd.concat([df,df_test],axis=0)

C:\Users\AJIT\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
ID_col = ['ID_code']
target_col = ["target"]
other_col=['Type'] #Test and Train Data set identifier
num_cols= list(set(list(data.columns))-set(ID_col)-set(target_col)-set(other_col))

In [5]:
#Target variable is also a categorical so convert it
data["target"] =data["target"].astype('str')

train=data[data['Type']=='Train']
test=data[data['Type']=='Test']

train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
Train, Validate = train[train['is_train']==True], train[train['is_train']==False]
 

C:\Users\AJIT\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [6]:
features=list(set(list(data.columns))-set(ID_col)-set(target_col)-set(other_col))

In [44]:
x_train = Train[list(features)].values
y_train = Train["target"].values
x_validate = Validate[list(features)].values
y_validate = Validate["target"].values
x_test=test[list(features)].values

In [8]:
y_validate

array(['0.0', '1.0', '0.0', ..., '0.0', '0.0', '0.0'], dtype=object)

### PCA

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_validate = scaler.transform(x_validate)

In [10]:
x_test= scaler.transform(x_test)

In [11]:
from sklearn.decomposition import PCA
pca = PCA(.80)

In [12]:
pca.fit(x_train)
x_train = pca.transform(x_train)
x_validate= pca.transform(x_validate)

In [13]:
x_test= pca.transform(x_test)

In [14]:
x_train.shape

(149941, 158)

### RFECV

In [71]:
#https://chrisalbon.com/machine_learning/feature_selection/recursive_feature_elimination/
#https://www.scikit-yb.org/en/latest/api/features/rfecv.html
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

In [72]:
model = LogisticRegression()
rfecv = RFECV(estimator=model,scoring="roc_auc")

In [85]:
#rfecv.fit(x_train, y_train)
#rfecv.transform(x_train)

In [ ]:
rfecv.support_

In [ ]:
cols = [2,9,16,26,69,72,81,84,93,94,101]
X_train.drop(X_train.columns[cols],axis=1,inplace=True)
X_test.drop(X_test.columns[cols],axis=1,inplace=True)

### Feature Selection using RF

In [8]:
#https://chrisalbon.com/machine_learning/trees_and_forests/feature_selection_using_random_forest/
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [35]:
clf = RandomForestClassifier(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(x_train.columns, clf.feature_importances_):
    print(feature)

C:\Users\AJIT\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


('var_17', 0.0038523891835451378)
('var_112', 0.004420889747898919)
('var_95', 0.0050224418003197956)
('var_12', 0.010838638899707782)
('var_165', 0.00706039769327597)
('var_167', 0.004833586933495103)
('var_56', 0.005545660887508649)
('var_6', 0.0066739677838199025)
('var_162', 0.004751954417536364)
('var_127', 0.0049128477797281845)
('var_91', 0.0062005155063363)
('var_58', 0.004141005304234164)
('var_131', 0.0045084133872987245)
('var_51', 0.0052508882270188)
('var_76', 0.0076667902893037005)
('var_194', 0.004967379143783622)
('var_124', 0.003759844579299526)
('var_89', 0.00514659720192982)
('var_54', 0.0037186841909330167)
('var_106', 0.00461405116912883)
('var_188', 0.004845065755632463)
('var_104', 0.003782856765853806)
('var_187', 0.004703560802065834)
('var_93', 0.005097544600971579)
('var_182', 0.0046084782593454904)
('var_96', 0.004105328628922418)
('var_38', 0.0037595533225658366)
('var_29', 0.003652709546923026)
('var_20', 0.004056215616840207)
('var_121', 0.005628842282190

In [36]:
sfm = SelectFromModel(clf, threshold=0.004)
sfm.fit(x_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
        max_features=None, norm_order=1, prefit=False, threshold=0.004)

In [37]:
for feature_list_index in sfm.get_support(indices=True):
    print(x_train.columns[feature_list_index])

var_112
var_95
var_12
var_165
var_167
var_56
var_6
var_162
var_127
var_91
var_58
var_131
var_51
var_76
var_194
var_89
var_106
var_188
var_187
var_93
var_182
var_96
var_20
var_121
var_79
var_163
var_126
var_170
var_43
var_80
var_87
var_52
var_175
var_19
var_156
var_130
var_174
var_147
var_197
var_108
var_85
var_63
var_59
var_90
var_148
var_99
var_122
var_150
var_193
var_183
var_184
var_10
var_149
var_74
var_40
var_97
var_49
var_33
var_101
var_152
var_180
var_146
var_139
var_151
var_115
var_11
var_164
var_143
var_81
var_166
var_153
var_117
var_34
var_50
var_109
var_119
var_113
var_141
var_75
var_5
var_4
var_172
var_125
var_135
var_137
var_78
var_181
var_154
var_196
var_132
var_41
var_31
var_114
var_32
var_123
var_36
var_69
var_13
var_128
var_195
var_9
var_26
var_169
var_71
var_22
var_199
var_118
var_86
var_21
var_107
var_173
var_179
var_35
var_92
var_177
var_77
var_111
var_3
var_23
var_186
var_2
var_102
var_110
var_105
var_28
var_16
var_191
var_168
var_65
var_70
var_198
var_190
var_94
va

In [38]:
X_train = sfm.transform(x_train)
X_validate = sfm.transform(x_validate)

In [39]:
X_train.shape

(150044, 156)

In [ ]:
X_validate = sfm.transform(X_validate)

### Bagging meta-estimator

In [85]:
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
bme= BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
bme.fit(x_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=None, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [86]:
probs = bme.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.803


### Random Forest

In [69]:
random.seed(100)
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

C:\Users\AJIT\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [70]:
probs = rf.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.701


### GBM

In [60]:
gbm= GradientBoostingClassifier()
gbm.fit(x_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [76]:
probs = gbm.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.859


In [66]:
final_status = gbm.predict_proba(x_test)
df_test["target"]=final_status[:,1]
df_test.to_csv("gbm.csv",columns=['ID_code','target'])

### Ada Boost

In [67]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(x_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [74]:
probs = ada.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.856


In [78]:
final_status = ada.predict_proba(x_test)
df_test["target"]=final_status[:,1]
df_test.to_csv("ada.csv",columns=['ID_code','target'])

### Light GBM

In [76]:
import lightgbm as lgb
lg = lgb.LGBMClassifier(silent=False)
lg.fit(x_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [77]:
probs = lg.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.865


In [83]:
final_status = lg.predict_proba(x_test)
df_test["target"]=final_status[:,1]
df_test.to_csv("lg.csv",columns=['ID_code','target'])

### Xg Boost

In [75]:
import xgboost as xgb
xg=xgb.XGBClassifier()
xg.fit(x_train, y_train)

KeyboardInterrupt: 

In [89]:
probs = xg.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.858


### Cat Boost

In [52]:
import catboost as cb
cbo=cb.CatBoostClassifier()
clf = cb.CatBoostClassifier(iterations=3000)
clf.fit(x_train,y_train)

Learning rate set to 0.025994
0:	learn: 0.6704199	total: 301ms	remaining: 15m 2s
1:	learn: 0.6490755	total: 646ms	remaining: 16m 8s
2:	learn: 0.6290141	total: 921ms	remaining: 15m 20s
3:	learn: 0.6101842	total: 1.23s	remaining: 15m 22s
4:	learn: 0.5925504	total: 1.52s	remaining: 15m 13s
5:	learn: 0.5762434	total: 1.8s	remaining: 15m 1s
6:	learn: 0.5604423	total: 2.09s	remaining: 14m 53s
7:	learn: 0.5459075	total: 2.33s	remaining: 14m 33s
8:	learn: 0.5322107	total: 2.6s	remaining: 14m 24s
9:	learn: 0.5191522	total: 2.86s	remaining: 14m 14s
10:	learn: 0.5073278	total: 3.14s	remaining: 14m 12s
11:	learn: 0.4956903	total: 3.42s	remaining: 14m 10s
12:	learn: 0.4852333	total: 3.69s	remaining: 14m 7s
13:	learn: 0.4751588	total: 3.96s	remaining: 14m 4s
14:	learn: 0.4657619	total: 4.22s	remaining: 14m
15:	learn: 0.4568525	total: 4.48s	remaining: 13m 55s
16:	learn: 0.4484439	total: 4.77s	remaining: 13m 57s
17:	learn: 0.4409169	total: 5.06s	remaining: 13m 58s
18:	learn: 0.4333802	total: 5.35s	rem

155:	learn: 0.2804051	total: 41.2s	remaining: 12m 31s
156:	learn: 0.2802077	total: 41.5s	remaining: 12m 30s
157:	learn: 0.2800190	total: 41.7s	remaining: 12m 29s
158:	learn: 0.2798151	total: 41.9s	remaining: 12m 27s
159:	learn: 0.2796168	total: 42.1s	remaining: 12m 26s
160:	learn: 0.2794178	total: 42.3s	remaining: 12m 25s
161:	learn: 0.2792212	total: 42.5s	remaining: 12m 23s
162:	learn: 0.2790293	total: 42.7s	remaining: 12m 22s
163:	learn: 0.2788510	total: 42.9s	remaining: 12m 21s
164:	learn: 0.2786663	total: 43.1s	remaining: 12m 20s
165:	learn: 0.2784756	total: 43.4s	remaining: 12m 20s
166:	learn: 0.2782885	total: 43.7s	remaining: 12m 20s
167:	learn: 0.2781133	total: 43.9s	remaining: 12m 20s
168:	learn: 0.2779245	total: 44.2s	remaining: 12m 20s
169:	learn: 0.2777134	total: 44.5s	remaining: 12m 20s
170:	learn: 0.2775116	total: 44.8s	remaining: 12m 20s
171:	learn: 0.2773253	total: 45s	remaining: 12m 19s
172:	learn: 0.2771410	total: 45.3s	remaining: 12m 19s
173:	learn: 0.2769467	total: 4

308:	learn: 0.2571996	total: 1m 22s	remaining: 11m 59s
309:	learn: 0.2571009	total: 1m 22s	remaining: 11m 59s
310:	learn: 0.2569829	total: 1m 23s	remaining: 11m 59s
311:	learn: 0.2568719	total: 1m 23s	remaining: 11m 59s
312:	learn: 0.2567554	total: 1m 23s	remaining: 12m
313:	learn: 0.2566315	total: 1m 24s	remaining: 12m
314:	learn: 0.2565067	total: 1m 24s	remaining: 12m
315:	learn: 0.2563746	total: 1m 24s	remaining: 12m
316:	learn: 0.2562614	total: 1m 25s	remaining: 11m 59s
317:	learn: 0.2561568	total: 1m 25s	remaining: 11m 59s
318:	learn: 0.2560473	total: 1m 25s	remaining: 11m 58s
319:	learn: 0.2559362	total: 1m 25s	remaining: 11m 58s
320:	learn: 0.2558166	total: 1m 26s	remaining: 11m 58s
321:	learn: 0.2557034	total: 1m 26s	remaining: 11m 57s
322:	learn: 0.2555861	total: 1m 26s	remaining: 11m 57s
323:	learn: 0.2554745	total: 1m 26s	remaining: 11m 57s
324:	learn: 0.2553521	total: 1m 27s	remaining: 11m 58s
325:	learn: 0.2552371	total: 1m 27s	remaining: 11m 58s
326:	learn: 0.2551302	tota

459:	learn: 0.2424505	total: 2m 12s	remaining: 12m 10s
460:	learn: 0.2423767	total: 2m 12s	remaining: 12m 10s
461:	learn: 0.2422872	total: 2m 13s	remaining: 12m 10s
462:	learn: 0.2422023	total: 2m 13s	remaining: 12m 10s
463:	learn: 0.2421271	total: 2m 13s	remaining: 12m 10s
464:	learn: 0.2420542	total: 2m 13s	remaining: 12m 10s
465:	learn: 0.2419667	total: 2m 14s	remaining: 12m 10s
466:	learn: 0.2418916	total: 2m 14s	remaining: 12m 10s
467:	learn: 0.2418022	total: 2m 15s	remaining: 12m 10s
468:	learn: 0.2417193	total: 2m 15s	remaining: 12m 10s
469:	learn: 0.2416424	total: 2m 15s	remaining: 12m 9s
470:	learn: 0.2415624	total: 2m 15s	remaining: 12m 9s
471:	learn: 0.2414843	total: 2m 16s	remaining: 12m 8s
472:	learn: 0.2414068	total: 2m 16s	remaining: 12m 8s
473:	learn: 0.2413273	total: 2m 16s	remaining: 12m 7s
474:	learn: 0.2412497	total: 2m 16s	remaining: 12m 7s
475:	learn: 0.2411787	total: 2m 17s	remaining: 12m 6s
476:	learn: 0.2410930	total: 2m 17s	remaining: 12m 6s
477:	learn: 0.2410

610:	learn: 0.2316836	total: 2m 59s	remaining: 11m 41s
611:	learn: 0.2316207	total: 2m 59s	remaining: 11m 41s
612:	learn: 0.2315602	total: 2m 59s	remaining: 11m 40s
613:	learn: 0.2314988	total: 3m	remaining: 11m 40s
614:	learn: 0.2314274	total: 3m	remaining: 11m 39s
615:	learn: 0.2313583	total: 3m	remaining: 11m 39s
616:	learn: 0.2312976	total: 3m	remaining: 11m 38s
617:	learn: 0.2312305	total: 3m 1s	remaining: 11m 38s
618:	learn: 0.2311682	total: 3m 1s	remaining: 11m 37s
619:	learn: 0.2311032	total: 3m 1s	remaining: 11m 37s
620:	learn: 0.2310407	total: 3m 2s	remaining: 11m 37s
621:	learn: 0.2309763	total: 3m 2s	remaining: 11m 37s
622:	learn: 0.2309047	total: 3m 2s	remaining: 11m 37s
623:	learn: 0.2308491	total: 3m 2s	remaining: 11m 36s
624:	learn: 0.2307912	total: 3m 3s	remaining: 11m 36s
625:	learn: 0.2307267	total: 3m 3s	remaining: 11m 35s
626:	learn: 0.2306561	total: 3m 3s	remaining: 11m 35s
627:	learn: 0.2305887	total: 3m 3s	remaining: 11m 34s
628:	learn: 0.2305210	total: 3m 4s	re

...

2265:	learn: 0.1817932	total: 11m	remaining: 3m 33s
2266:	learn: 0.1817753	total: 11m	remaining: 3m 33s
2267:	learn: 0.1817538	total: 11m 1s	remaining: 3m 33s
2268:	learn: 0.1817386	total: 11m 1s	remaining: 3m 33s
2269:	learn: 0.1817253	total: 11m 1s	remaining: 3m 32s
2270:	learn: 0.1817106	total: 11m 1s	remaining: 3m 32s
2271:	learn: 0.1816957	total: 11m 1s	remaining: 3m 32s
2272:	learn: 0.1816798	total: 11m 2s	remaining: 3m 31s
2273:	learn: 0.1816611	total: 11m 2s	remaining: 3m 31s
2274:	learn: 0.1816366	total: 11m 2s	remaining: 3m 31s
2275:	learn: 0.1816185	total: 11m 2s	remaining: 3m 30s
2276:	learn: 0.1816014	total: 11m 3s	remaining: 3m 30s
2277:	learn: 0.1815845	total: 11m 3s	remaining: 3m 30s
2278:	learn: 0.1815684	total: 11m 3s	remaining: 3m 29s
2279:	learn: 0.1815496	total: 11m 3s	remaining: 3m 29s
2280:	learn: 0.1815326	total: 11m 4s	remaining: 3m 29s
2281:	learn: 0.1815153	total: 11m 4s	remaining: 3m 29s
2282:	learn: 0.1815028	total: 11m 4s	remaining: 3m 28s
2283:	learn: 0.1

2413:	learn: 0.1793643	total: 11m 35s	remaining: 2m 48s
2414:	learn: 0.1793495	total: 11m 35s	remaining: 2m 48s
2415:	learn: 0.1793325	total: 11m 35s	remaining: 2m 48s
2416:	learn: 0.1793190	total: 11m 35s	remaining: 2m 47s
2417:	learn: 0.1792989	total: 11m 36s	remaining: 2m 47s
2418:	learn: 0.1792812	total: 11m 36s	remaining: 2m 47s
2419:	learn: 0.1792636	total: 11m 36s	remaining: 2m 46s
2420:	learn: 0.1792480	total: 11m 36s	remaining: 2m 46s
2421:	learn: 0.1792301	total: 11m 37s	remaining: 2m 46s
2422:	learn: 0.1792151	total: 11m 37s	remaining: 2m 46s
2423:	learn: 0.1791988	total: 11m 37s	remaining: 2m 45s
2424:	learn: 0.1791837	total: 11m 37s	remaining: 2m 45s
2425:	learn: 0.1791699	total: 11m 38s	remaining: 2m 45s
2426:	learn: 0.1791519	total: 11m 38s	remaining: 2m 44s
2427:	learn: 0.1791336	total: 11m 38s	remaining: 2m 44s
2428:	learn: 0.1791218	total: 11m 38s	remaining: 2m 44s
2429:	learn: 0.1791089	total: 11m 39s	remaining: 2m 43s
2430:	learn: 0.1790918	total: 11m 39s	remaining:

2561:	learn: 0.1770402	total: 12m 10s	remaining: 2m 4s
2562:	learn: 0.1770278	total: 12m 10s	remaining: 2m 4s
2563:	learn: 0.1770133	total: 12m 11s	remaining: 2m 4s
2564:	learn: 0.1769972	total: 12m 11s	remaining: 2m 4s
2565:	learn: 0.1769773	total: 12m 11s	remaining: 2m 3s
2566:	learn: 0.1769596	total: 12m 11s	remaining: 2m 3s
2567:	learn: 0.1769415	total: 12m 12s	remaining: 2m 3s
2568:	learn: 0.1769291	total: 12m 12s	remaining: 2m 2s
2569:	learn: 0.1769133	total: 12m 12s	remaining: 2m 2s
2570:	learn: 0.1768988	total: 12m 12s	remaining: 2m 2s
2571:	learn: 0.1768801	total: 12m 13s	remaining: 2m 1s
2572:	learn: 0.1768636	total: 12m 13s	remaining: 2m 1s
2573:	learn: 0.1768434	total: 12m 13s	remaining: 2m 1s
2574:	learn: 0.1768324	total: 12m 14s	remaining: 2m 1s
2575:	learn: 0.1768143	total: 12m 14s	remaining: 2m
2576:	learn: 0.1768031	total: 12m 14s	remaining: 2m
2577:	learn: 0.1767935	total: 12m 15s	remaining: 2m
2578:	learn: 0.1767787	total: 12m 15s	remaining: 2m
2579:	learn: 0.1767598

2708:	learn: 0.1748556	total: 12m 47s	remaining: 1m 22s
2709:	learn: 0.1748421	total: 12m 47s	remaining: 1m 22s
2710:	learn: 0.1748300	total: 12m 47s	remaining: 1m 21s
2711:	learn: 0.1748175	total: 12m 48s	remaining: 1m 21s
2712:	learn: 0.1747988	total: 12m 48s	remaining: 1m 21s
2713:	learn: 0.1747836	total: 12m 48s	remaining: 1m 20s
2714:	learn: 0.1747675	total: 12m 48s	remaining: 1m 20s
2715:	learn: 0.1747551	total: 12m 49s	remaining: 1m 20s
2716:	learn: 0.1747412	total: 12m 49s	remaining: 1m 20s
2717:	learn: 0.1747259	total: 12m 49s	remaining: 1m 19s
2718:	learn: 0.1747110	total: 12m 49s	remaining: 1m 19s
2719:	learn: 0.1746997	total: 12m 49s	remaining: 1m 19s
2720:	learn: 0.1746863	total: 12m 50s	remaining: 1m 18s
2721:	learn: 0.1746667	total: 12m 50s	remaining: 1m 18s
2722:	learn: 0.1746532	total: 12m 50s	remaining: 1m 18s
2723:	learn: 0.1746353	total: 12m 50s	remaining: 1m 18s
2724:	learn: 0.1746229	total: 12m 51s	remaining: 1m 17s
2725:	learn: 0.1746048	total: 12m 51s	remaining:

2858:	learn: 0.1726753	total: 13m 22s	remaining: 39.6s
2859:	learn: 0.1726600	total: 13m 22s	remaining: 39.3s
2860:	learn: 0.1726427	total: 13m 22s	remaining: 39s
2861:	learn: 0.1726274	total: 13m 22s	remaining: 38.7s
2862:	learn: 0.1726094	total: 13m 23s	remaining: 38.4s
2863:	learn: 0.1725952	total: 13m 23s	remaining: 38.1s
2864:	learn: 0.1725810	total: 13m 23s	remaining: 37.9s
2865:	learn: 0.1725647	total: 13m 23s	remaining: 37.6s
2866:	learn: 0.1725458	total: 13m 23s	remaining: 37.3s
2867:	learn: 0.1725308	total: 13m 24s	remaining: 37s
2868:	learn: 0.1725189	total: 13m 24s	remaining: 36.7s
2869:	learn: 0.1725086	total: 13m 24s	remaining: 36.4s
2870:	learn: 0.1724948	total: 13m 24s	remaining: 36.2s
2871:	learn: 0.1724823	total: 13m 25s	remaining: 35.9s
2872:	learn: 0.1724677	total: 13m 25s	remaining: 35.6s
2873:	learn: 0.1724510	total: 13m 25s	remaining: 35.3s
2874:	learn: 0.1724373	total: 13m 25s	remaining: 35s
2875:	learn: 0.1724269	total: 13m 25s	remaining: 34.7s
2876:	learn: 0.1

In [53]:
probs = clf.predict_proba(x_validate)
probs = probs[:,1]
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.897


### Tuning

In [8]:
params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [300]}

In [10]:
import catboost as cb
cb = cb.CatBoostClassifier()

In [11]:
from sklearn.model_selection import GridSearchCV
cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3)
cb_model.fit(x_train, y_train)

0:	learn: 0.6673290	total: 786ms	remaining: 3m 54s
1:	learn: 0.6432885	total: 961ms	remaining: 2m 23s
2:	learn: 0.6206508	total: 1.11s	remaining: 1m 50s
3:	learn: 0.5999720	total: 1.3s	remaining: 1m 36s
4:	learn: 0.5803764	total: 1.45s	remaining: 1m 25s
5:	learn: 0.5621461	total: 1.59s	remaining: 1m 18s
6:	learn: 0.5456060	total: 1.74s	remaining: 1m 12s
7:	learn: 0.5301269	total: 1.9s	remaining: 1m 9s
8:	learn: 0.5154000	total: 2.04s	remaining: 1m 5s
9:	learn: 0.5018636	total: 2.19s	remaining: 1m 3s
10:	learn: 0.4893998	total: 2.34s	remaining: 1m 1s
11:	learn: 0.4779940	total: 2.58s	remaining: 1m 1s
12:	learn: 0.4671532	total: 2.76s	remaining: 1m 1s
13:	learn: 0.4570898	total: 2.98s	remaining: 1m
14:	learn: 0.4476727	total: 3.36s	remaining: 1m 3s
15:	learn: 0.4386813	total: 3.48s	remaining: 1m 1s
16:	learn: 0.4305311	total: 3.71s	remaining: 1m 1s
17:	learn: 0.4227676	total: 3.83s	remaining: 1m
18:	learn: 0.4156298	total: 3.98s	remaining: 58.9s
19:	learn: 0.4090385	total: 4.09s	remainin

161:	learn: 0.2860080	total: 25s	remaining: 21.3s
162:	learn: 0.2858282	total: 25.1s	remaining: 21.1s
163:	learn: 0.2856602	total: 25.2s	remaining: 20.9s
164:	learn: 0.2854785	total: 25.4s	remaining: 20.8s
165:	learn: 0.2853036	total: 25.5s	remaining: 20.6s
166:	learn: 0.2851321	total: 25.6s	remaining: 20.4s
167:	learn: 0.2849721	total: 25.7s	remaining: 20.2s
168:	learn: 0.2847937	total: 25.8s	remaining: 20s
169:	learn: 0.2846280	total: 26s	remaining: 19.9s
170:	learn: 0.2844762	total: 26.1s	remaining: 19.7s
171:	learn: 0.2843190	total: 26.2s	remaining: 19.5s
172:	learn: 0.2841713	total: 26.4s	remaining: 19.4s
173:	learn: 0.2839930	total: 26.7s	remaining: 19.3s
174:	learn: 0.2838323	total: 26.9s	remaining: 19.2s
175:	learn: 0.2836582	total: 27.1s	remaining: 19.1s
176:	learn: 0.2834931	total: 27.2s	remaining: 18.9s
177:	learn: 0.2833260	total: 27.4s	remaining: 18.8s
178:	learn: 0.2831517	total: 27.6s	remaining: 18.7s
179:	learn: 0.2829776	total: 27.8s	remaining: 18.6s
180:	learn: 0.2828

22:	learn: 0.3909895	total: 3.09s	remaining: 37.2s
23:	learn: 0.3858919	total: 3.25s	remaining: 37.4s
24:	learn: 0.3813769	total: 3.44s	remaining: 37.9s
25:	learn: 0.3770524	total: 3.61s	remaining: 38.1s
26:	learn: 0.3728782	total: 3.75s	remaining: 37.9s
27:	learn: 0.3689083	total: 3.91s	remaining: 38s
28:	learn: 0.3652963	total: 4.09s	remaining: 38.2s
29:	learn: 0.3619367	total: 4.24s	remaining: 38.2s
30:	learn: 0.3587450	total: 4.4s	remaining: 38.2s
31:	learn: 0.3556297	total: 4.56s	remaining: 38.2s
32:	learn: 0.3527223	total: 4.72s	remaining: 38.2s
33:	learn: 0.3499094	total: 4.85s	remaining: 38s
34:	learn: 0.3473969	total: 5s	remaining: 37.9s
35:	learn: 0.3450520	total: 5.11s	remaining: 37.5s
36:	learn: 0.3427966	total: 5.24s	remaining: 37.2s
37:	learn: 0.3406734	total: 5.36s	remaining: 37s
38:	learn: 0.3386961	total: 5.48s	remaining: 36.7s
39:	learn: 0.3368708	total: 5.62s	remaining: 36.5s
40:	learn: 0.3350900	total: 5.75s	remaining: 36.4s
41:	learn: 0.3333521	total: 5.87s	remaini

184:	learn: 0.2820149	total: 24.1s	remaining: 15s
185:	learn: 0.2818439	total: 24.2s	remaining: 14.9s
186:	learn: 0.2816821	total: 24.4s	remaining: 14.7s
187:	learn: 0.2815152	total: 24.5s	remaining: 14.6s
188:	learn: 0.2813557	total: 24.6s	remaining: 14.5s
189:	learn: 0.2812053	total: 24.7s	remaining: 14.3s
190:	learn: 0.2810564	total: 24.8s	remaining: 14.2s
191:	learn: 0.2809194	total: 24.9s	remaining: 14s
192:	learn: 0.2807702	total: 25.1s	remaining: 13.9s
193:	learn: 0.2806090	total: 25.2s	remaining: 13.8s
194:	learn: 0.2804533	total: 25.3s	remaining: 13.6s
195:	learn: 0.2803044	total: 25.4s	remaining: 13.5s
196:	learn: 0.2801616	total: 25.5s	remaining: 13.4s
197:	learn: 0.2800036	total: 25.7s	remaining: 13.2s
198:	learn: 0.2798429	total: 25.8s	remaining: 13.1s
199:	learn: 0.2796883	total: 25.9s	remaining: 13s
200:	learn: 0.2795334	total: 26s	remaining: 12.8s
201:	learn: 0.2793848	total: 26.1s	remaining: 12.7s
202:	learn: 0.2792306	total: 26.3s	remaining: 12.6s
203:	learn: 0.279096

45:	learn: 0.3277232	total: 5.82s	remaining: 32.2s
46:	learn: 0.3264089	total: 5.93s	remaining: 31.9s
47:	learn: 0.3252416	total: 6.05s	remaining: 31.8s
48:	learn: 0.3241324	total: 6.16s	remaining: 31.6s
49:	learn: 0.3230734	total: 6.29s	remaining: 31.4s
50:	learn: 0.3221001	total: 6.41s	remaining: 31.3s
51:	learn: 0.3211353	total: 6.53s	remaining: 31.1s
52:	learn: 0.3201786	total: 6.63s	remaining: 30.9s
53:	learn: 0.3193241	total: 6.75s	remaining: 30.8s
54:	learn: 0.3185346	total: 6.87s	remaining: 30.6s
55:	learn: 0.3177297	total: 6.98s	remaining: 30.4s
56:	learn: 0.3169764	total: 7.11s	remaining: 30.3s
57:	learn: 0.3162074	total: 7.24s	remaining: 30.2s
58:	learn: 0.3155523	total: 7.35s	remaining: 30s
59:	learn: 0.3148054	total: 7.48s	remaining: 29.9s
60:	learn: 0.3141750	total: 7.58s	remaining: 29.7s
61:	learn: 0.3135447	total: 7.69s	remaining: 29.5s
62:	learn: 0.3129085	total: 7.82s	remaining: 29.4s
63:	learn: 0.3123239	total: 7.95s	remaining: 29.3s
64:	learn: 0.3117947	total: 8.06s

...

248:	learn: 0.2197400	total: 8m 22s	remaining: 1m 42s
249:	learn: 0.2195350	total: 8m 24s	remaining: 1m 40s
250:	learn: 0.2193241	total: 8m 26s	remaining: 1m 38s
251:	learn: 0.2191052	total: 8m 28s	remaining: 1m 36s
252:	learn: 0.2188856	total: 8m 30s	remaining: 1m 34s
253:	learn: 0.2186415	total: 8m 32s	remaining: 1m 32s
254:	learn: 0.2184276	total: 8m 34s	remaining: 1m 30s
255:	learn: 0.2182019	total: 8m 36s	remaining: 1m 28s
256:	learn: 0.2180134	total: 8m 38s	remaining: 1m 26s
257:	learn: 0.2177699	total: 8m 40s	remaining: 1m 24s
258:	learn: 0.2175668	total: 8m 42s	remaining: 1m 22s
259:	learn: 0.2173820	total: 8m 44s	remaining: 1m 20s
260:	learn: 0.2171726	total: 8m 46s	remaining: 1m 18s
261:	learn: 0.2169697	total: 8m 48s	remaining: 1m 16s
262:	learn: 0.2167083	total: 8m 50s	remaining: 1m 14s
263:	learn: 0.2164917	total: 8m 52s	remaining: 1m 12s
264:	learn: 0.2162550	total: 8m 54s	remaining: 1m 10s
265:	learn: 0.2160438	total: 8m 56s	remaining: 1m 8s
266:	learn: 0.2158294	total: 

104:	learn: 0.2637600	total: 3m 55s	remaining: 7m 17s
105:	learn: 0.2633338	total: 3m 57s	remaining: 7m 15s
106:	learn: 0.2629697	total: 3m 59s	remaining: 7m 12s
107:	learn: 0.2625655	total: 4m 2s	remaining: 7m 10s
108:	learn: 0.2622027	total: 4m 4s	remaining: 7m 7s
109:	learn: 0.2617950	total: 4m 6s	remaining: 7m 5s
110:	learn: 0.2613786	total: 4m 8s	remaining: 7m 3s
111:	learn: 0.2609745	total: 4m 10s	remaining: 7m
112:	learn: 0.2605534	total: 4m 12s	remaining: 6m 58s
113:	learn: 0.2601768	total: 4m 14s	remaining: 6m 55s
114:	learn: 0.2597918	total: 4m 16s	remaining: 6m 52s
115:	learn: 0.2593927	total: 4m 18s	remaining: 6m 50s
116:	learn: 0.2590126	total: 4m 20s	remaining: 6m 47s
117:	learn: 0.2585652	total: 4m 22s	remaining: 6m 45s
118:	learn: 0.2581984	total: 4m 24s	remaining: 6m 42s
119:	learn: 0.2578120	total: 4m 26s	remaining: 6m 40s
120:	learn: 0.2574232	total: 4m 28s	remaining: 6m 37s
121:	learn: 0.2570432	total: 4m 30s	remaining: 6m 34s
122:	learn: 0.2566407	total: 4m 32s	rem

257:	learn: 0.2179084	total: 9m 9s	remaining: 1m 29s
258:	learn: 0.2176538	total: 9m 12s	remaining: 1m 27s
259:	learn: 0.2174193	total: 9m 14s	remaining: 1m 25s
260:	learn: 0.2172379	total: 9m 16s	remaining: 1m 23s
261:	learn: 0.2170483	total: 9m 18s	remaining: 1m 20s
262:	learn: 0.2168508	total: 9m 19s	remaining: 1m 18s
263:	learn: 0.2166189	total: 9m 21s	remaining: 1m 16s
264:	learn: 0.2164230	total: 9m 23s	remaining: 1m 14s
265:	learn: 0.2161913	total: 9m 25s	remaining: 1m 12s
266:	learn: 0.2159826	total: 9m 27s	remaining: 1m 10s
267:	learn: 0.2157885	total: 9m 29s	remaining: 1m 8s
268:	learn: 0.2155853	total: 9m 31s	remaining: 1m 5s
269:	learn: 0.2153438	total: 9m 34s	remaining: 1m 3s
270:	learn: 0.2151223	total: 9m 36s	remaining: 1m 1s
271:	learn: 0.2148855	total: 9m 38s	remaining: 59.5s
272:	learn: 0.2146619	total: 9m 40s	remaining: 57.4s
273:	learn: 0.2144374	total: 9m 42s	remaining: 55.3s
274:	learn: 0.2142190	total: 9m 44s	remaining: 53.1s
275:	learn: 0.2140351	total: 9m 46s	r

113:	learn: 0.2613391	total: 3m 57s	remaining: 6m 27s
114:	learn: 0.2609594	total: 4m	remaining: 6m 26s
115:	learn: 0.2605839	total: 4m 2s	remaining: 6m 24s
116:	learn: 0.2601975	total: 4m 4s	remaining: 6m 22s
117:	learn: 0.2597846	total: 4m 7s	remaining: 6m 21s
118:	learn: 0.2593878	total: 4m 9s	remaining: 6m 19s
119:	learn: 0.2589645	total: 4m 12s	remaining: 6m 18s
120:	learn: 0.2585917	total: 4m 14s	remaining: 6m 17s
121:	learn: 0.2582117	total: 4m 17s	remaining: 6m 15s
122:	learn: 0.2578027	total: 4m 19s	remaining: 6m 12s
123:	learn: 0.2574447	total: 4m 21s	remaining: 6m 10s
124:	learn: 0.2570814	total: 4m 23s	remaining: 6m 8s
125:	learn: 0.2567266	total: 4m 25s	remaining: 6m 6s
126:	learn: 0.2563819	total: 4m 27s	remaining: 6m 4s
127:	learn: 0.2560433	total: 4m 29s	remaining: 6m 2s
128:	learn: 0.2556921	total: 4m 31s	remaining: 6m
129:	learn: 0.2553530	total: 4m 33s	remaining: 5m 58s
130:	learn: 0.2549867	total: 4m 35s	remaining: 5m 55s
131:	learn: 0.2546092	total: 4m 37s	remainin

KeyboardInterrupt: 

In [54]:
final_status = clf.predict_proba(x_test)
df_test["target"]=final_status[:,1]
df_test.to_csv("clf.csv",columns=['ID_code','target'])

### H2O

In [51]:
x_train=pd.DataFrame(x_train)
y_train=pd.DataFrame(y_train)
x_validate=pd.DataFrame(x_validate)
y_validate=pd.DataFrame(y_validate)
x_test=pd.DataFrame(x_test)

In [52]:
df_train=pd.concat([x_train,y_train],axis=1)
df_val=pd.concat([x_validate,y_validate],axis=1)


In [53]:
df_train.to_csv("df_train.csv")
df_val.to_csv("df_val.csv")

In [54]:
import h2o
import numpy as np
import math
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
h2o.init(nthreads=-1, strict_version_check=True)
## optional: connect to a running H2O cluster
#h2o.init(ip="mycluster", port=55555)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,34 mins 45 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.4
H2O cluster version age:,1 month and 17 days
H2O cluster name:,H2O_from_python_AJIT_gxc13r
H2O cluster total nodes:,1
H2O cluster free memory:,532 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [55]:
## 'path' can point to a local file, hdfs, s3, nfs, Hive, directories, etc.
df = h2o.import_file(path = "C:\\Users\\AJIT\\Desktop\\df_train.csv")
df_val = h2o.import_file(path ="C:\\Users\\AJIT\\Desktop\\df_val.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [56]:
df.head()

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,C101,C102,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112,C113,C114,C115,C116,C117,C118,C119,C120,C121,C122,C123,C124,C125,C126,C127,C128,C129,C130,C131,C132,C133,C134,C135,C136,C137,C138,C139,C140,C141,C142,C143,C144,C145,C146,C147,C148,C149,C150,C151,C152,C153,C154,C155,C156,C157,C158,C159,C160,C161,C162,C163,C164,C165,C166,C167,C168,C169,C170,C171,C172,C173,C174,C175,C176,C177,C178,C179,C180,C181,C182,C183,C184,C185,C186,C187,C188,C189,C190,C191,C192,C193,C194,C195,C196,C197,C198,C199,C200
nan,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198
0,31.4045,8.6989,6.7806,18.3816,-19.7159,12.9041,5.8056,17.9244,-3.5189,12.1749,3.8743,-0.2154,0.5857,9.3225,3.9642,15.4462,16.1828,-4.5213,1.691,9.6508,11.2668,9.7268,17.1554,11.9081,-1.9245,-1.0914,7.8784,-13.4221,24.3627,1.3675,-3.4132,13.6468,3.1364,0.8897,14.0137,9.9913,-5.1488,8.3694,14.71,12.0039,5.7487,8.7989,29.046,-0.694,8.9255,3.6159,13.4325,12.9194,5.1187,8.86,2.8279,4.6667,4.9306,14.3831,2.524,-6.7863,11.2663,9.5981,8.3307,0.6764,12.0446,16.2572,18.5177,16.0094,3.1345,-4.7645,-4.92,-0.3085,21.6374,0.5663,-1.7069,12.1983,0.1812,1.4403,10.5785,-4.701,10.535,6.7602,16.0283,4.0288,16.8911,2.7004,11.5423,5.3822,1.7202,15.6599,9.5319,-13.1324,7.7423,11.0924,2.9423,6.676,5.3133,22.4321,-0.7474,4.284,3.0657,-9.2834,5.4879,43.1127,0.5745,11.5767,29.4547,2.0771,18.6266,3.9141,-0.6975,25.8398,-8.4051,9.0056,11.5659,6.5674,11.8411,4.959,5.1726,9.4763,-0.4073,30.7133,-5.2387,-0.4851,13.7241,11.192,14.8421,18.5227,6.5199,16.4597,19.0614,14.5691,-16.4727,5.0384,13.8571,8.9642,-3.8766,15.4576,2.1743,17.5743,2.9252,10.78,4.4354,-14.2136,4.2218,0.3587,2.5516,2.4716,-10.1245,13.8372,23.4104,14.1104,10.4838,5.9226,-21.4494,12.6644,26.5376,14.8322,5.1999,2.0323,1.5093,12.8287,17.6054,7.8871,21.4669,5.1736,2.0183,-0.7338,5.0109,-8.4254,-2.3978,8.5635,1.6573,5.3253,5.747,18.5618,6.0454,0.7754,5.8764,-5.6011,8.6748,-7.2393,-7.017,-2.344,7.9984,20.8773,16.6964,3.1531,14.3691,9.0164,12.7803,16.2191,5.093,10.1166,5.9965,2.5791,9.5426,13.3102,15.5752,11.4607,7.3746,11.6418,3.1821
1,18.1577,8.6086,6.8852,7.0529,-15.9319,7.8755,5.4007,-4.4223,-7.3477,12.2419,3.6783,-1.3147,-0.3566,-13.5901,7.7214,16.5346,8.0109,-5.485,10.9516,5.7582,19.3219,9.5018,32.0888,13.8588,0.8194,1.9518,8.1267,-11.51,13.0858,0.3683,0.6939,2.5462,2.5837,8.2736,14.0239,10.9631,-11.7684,11.0738,15.1866,12.5942,13.8275,5.4345,13.2256,-4.1083,11.5006,3.8208,13.891,11.8343,5.6208,20.1068,-1.9721,10.816,5.0464,6.9779,3.6159,-4.1473,12.2806,3.8821,3.6937,2.7995,11.7842,26.6881,20.6294,31.5899,-8.9921,5.5378,3.1468,-9.2987,40.5632,0.5134,12.2301,13.7814,-1.2722,1.7093,0.7399,2.2302,3.4287,7.1051,34.4014,4.2578,19.371,3.2003,10.6261,7.0118,11.0752,16.1622,1.5365,-9.6953,10.4968,8.0905,-4.821,-5.0121,5.8683,7.9344,-8.0946,7.8,-14.91,7.0433,5.7999,7.7841,8.4135,9.8775,9.7797,1.7944,16.5338,5.3523,9.3192,22.5441,-5.5937,9.4317,8.9231,7.0513,7.6543,-7.2417,1.9568,-13.695,-2.5586,28.5708,-11.1147,8.5426,16.5721,11.3702,11.7569,15.430

In [37]:
response = "C160"
df[response] = df[response].asfactor()           
df_val[response] = df_val[response].asfactor()           
predictors = df.columns
predictors[1:156]

['C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26',
 'C27',
 'C28',
 'C29',
 'C30',
 'C31',
 'C32',
 'C33',
 'C34',
 'C35',
 'C36',
 'C37',
 'C38',
 'C39',
 'C40',
 'C41',
 'C42',
 'C43',
 'C44',
 'C45',
 'C46',
 'C47',
 'C48',
 'C49',
 'C50',
 'C51',
 'C52',
 'C53',
 'C54',
 'C55',
 'C56',
 'C57',
 'C58',
 'C59',
 'C60',
 'C61',
 'C62',
 'C63',
 'C64',
 'C65',
 'C66',
 'C67',
 'C68',
 'C69',
 'C70',
 'C71',
 'C72',
 'C73',
 'C74',
 'C75',
 'C76',
 'C77',
 'C78',
 'C79',
 'C80',
 'C81',
 'C82',
 'C83',
 'C84',
 'C85',
 'C86',
 'C87',
 'C88',
 'C89',
 'C90',
 'C91',
 'C92',
 'C93',
 'C94',
 'C95',
 'C96',
 'C97',
 'C98',
 'C99',
 'C100',
 'C101',
 'C102',
 'C103',
 'C104',
 'C105',
 'C106',
 'C107',
 'C108',
 'C109',
 'C110',
 'C111',
 'C112',
 'C113',
 'C114',
 'C115',
 'C116',
 'C117',
 'C118',
 'C119',
 'C120',
 'C121',
 'C122',
 'C123',
 'C124',

In [38]:
gbm = H2OGradientBoostingEstimator()
gbm.train(x=predictors, y=response, training_frame=df)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [40]:
perf = gbm.model_performance(df_val)
perf.auc()

0.8562849093618705

### Random Forest

In [41]:
#We only provide the required parameters, everything else is default
rf = h2o.estimators.random_forest.H2ORandomForestEstimator()
rf.train(x=predictors, y=response, training_frame=df)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [42]:
perf = rf.model_performance(df_val)
perf.auc()

0.837884031467246